In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal

In [ ]:
data_source_train=pd.read_csv(r'data_source_train.csv')
data_source_test=pd.read_csv(r'data_source_test.csv')


data_target_labeled=pd.read_csv(r'data_target_labeled.csv')
data_target_unlabeled=pd.read_csv(r'data_target_unlabeled.csv')
data_target_test=pd.read_csv(r'data_target_test.csv')




In [ ]:
#(a)
model= AdaBoostClassifier()
model.fit(data_source_train[['feat_0','feat_1']],data_source_train['class'])
print(accuracy_score(data_source_test['class'], model.predict(data_source_test[['feat_0','feat_1']])))

In [ ]:
#(b)
print(accuracy_score(data_target_test['class'], model.predict(data_target_test[['feat_0','feat_1']])))

In [ ]:
#(c)
model= AdaBoostClassifier()
model.fit(data_target_labeled[['feat_0','feat_1']],data_target_labeled['class'])
print(accuracy_score(data_target_test['class'], model.predict(data_target_test[['feat_0','feat_1']])))

In [ ]:
#(d)
model= AdaBoostClassifier()
model.fit(data_source_train[['feat_0','feat_1']].append(data_target_labeled[['feat_0','feat_1']]),data_source_train['class'].append(data_target_labeled['class']))
print(accuracy_score(data_target_test['class'], model.predict(data_target_test[['feat_0','feat_1']])))

#(e)
The classifier performed relatively well when it was trained and tested on the source domain.

The classifier performed poorly in the target domain when it was trained only with the limted number of labled data in the target domain, but the performance enhanced when we added the source training points to our training procedure.

We notice that the classifier performed equivalently when it was trained using the source training only, and when it was trained using the source training with extra labeled data in the target domanin, and this probably because the number of labeled points in the target domain is very few and it will not make a difference in enhancing the model's performace on the target domanin

In [ ]:
#################

In [ ]:
#(f)(i)
gm = GaussianMixture(n_components=2,random_state=0).fit(data_source_train[['feat_0','feat_1']].append(data_target_unlabeled[['feat_0','feat_1']].append(data_target_labeled[['feat_0','feat_1']])))
print(gm.means_)
print('----------------')
print(gm.covariances_)

In [ ]:
#(ii)
#source
print('for source')
gm = GaussianMixture(n_components=1,random_state=0).fit(data_source_train[['feat_0','feat_1']])
mu_s=gm.means_
print(mu_s)
print('----------------')
var_s=gm.covariances_
print(var_s)

#target
print('for target')
gm = GaussianMixture(n_components=1,random_state=0).fit(data_target_unlabeled[['feat_0','feat_1']].append(data_target_labeled[['feat_0','feat_1']]))
mu_t=gm.means_
print(mu_t)
print('----------------')
var_t=gm.covariances_
print(var_t)


The second approach yelds a better reults because the two distributions P_s(x) and P_t(x) are different due to the covariate shift. So, it is more likely that we will get a more accurate results when the parameters on each domain are estimated seperately.

In [ ]:
#(g)
sample_weights = np.array([(multivariate_normal.pdf(x, mean=[-1.05830525, -0.45760988], cov=[[ 0.76377201 , 0.30418792],[ 0.30418792 ,19.02092692]]) 
                           / multivariate_normal.pdf(x, mean=[1.14916266, 1.09731636], cov=[[ 3.70506986, -0.08005567],[-0.08005567 , 0.65845626]]))
                           for x in np.array(data_source_train[['feat_0','feat_1']])])


In [ ]:
model= AdaBoostClassifier()
model.fit(data_source_train[['feat_0','feat_1']],data_source_train['class'], sample_weights)
print(accuracy_score(data_target_test['class'], model.predict(data_target_test[['feat_0','feat_1']])))

In [ ]:
#with the unwighted target lableded data
model= AdaBoostClassifier()
model.fit(data_source_train[['feat_0','feat_1']].append(data_target_labeled[['feat_0','feat_1']]),data_source_train['class'].append(data_target_labeled['class']), np.append(sample_weights,[1,1]))
print(accuracy_score(data_target_test['class'], model.predict(data_target_test[['feat_0','feat_1']])))

The transfer learing approach performed better than the supervised learning approach because the source training points have been weighted to compenstate for the change in the density function in the two domains. The supervised approach trained the classifiers using the few target training points and the original source training point without any correction factor, and hence it is expected to perform worse that the transfer learning approach.

We also notice that the best accuracy achieved was when the weighted  source training data and the few labled target training data were all utlized.

In [ ]:
#end